# Master Energy Systems MILP Case Study

__author__ = "Rahul Kakodkar"
__copyright__ = "Copyright 2023, Multi-parametric Optimization & Control Lab"
__credits__ = ["Rahul Kakodkar", "Efstratios N. Pistikopoulos"]
__license__ = "MIT"
__version__ = "1.0.5"
__maintainer__ = "Rahul Kakodkar"
__email__ = "cacodcar@tamu.edu"
__status__ = "Production"


The following case study considers three locations along with their fluctuating local demand, weather profiles (capacity factors), and resource (natural gas) prices.

The following processes are considered - 

- Modular nuclear reactors (ASMR)
- Proton Exchange Membrane Electrolyzers (PEM)
- Hydrogen Storage (H2_L_c/H2_L_d)
- Natural Gas Combined Cycle with 95% carbon capture (NGCC)
- Pumped Storage Hydropower (PSH/PSH_discharge)
- Lithium-ion batteries (LiI/LiI_discharge)
- Solar Photovoltaics (PV) 
- Wind Farms (WF)
- Steam Methane Reforming (SMR)
- SMR + Carbon Capture Utilization and Storage (SMRH)
- Hydrogen Fuel Cells (H2FC)
- Direct Air Capture (DAC)
- Offshore Aquifers (AQoff)

The stated processes utilize the following resources:

- Methane (CH4)
- Carbon Dioxide (CO2)
- CO2 in aquifer (CO2_AQoff)
- CO2 Captured from air (CO2_DAC)
- Vented CO2 (CO2_Vent)
- Hydrogen (H2)
- Water (H2O)
- Stored hydrogen (H2_L)
- Power stored in LiI (LiI_Power)
- Oxygen (O2)
- Power stored in PSH (PSH_Power)
- Power
- Solar
- Uranium
- Wind


For the full formulation refer: https://www.mdpi.com/article/10.3390/pr12030469/s1

## Import Modules

In [1]:
import sys
sys.path.append('src')

In [2]:

from energiapy.components import *
from energiapy.plot import *
from energiapy.aggregation import *
import pandas
from itertools import product
from matplotlib import rc
import matplotlib.pyplot as plt
from energiapy.model.solve import solve
from energiapy.utils.nsrdb_utils import fetch_nsrdb_data
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.model.bounds import CapacityBounds
from energiapy.utils.data_utils import get_data, make_henry_price_df, remove_outliers, load_results, calculate_hourly
import numpy
from energiapy.conversion.photovoltaic import solar_power_output
from energiapy.conversion.windmill import wind_power_output


## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston
- Natural Gas prices: Henry Hub 


In [3]:
horizon = 1

### Weather data

The fetch_nsrdb function accesses the [National Solar Radiation Database (NSRDB)](https://nsrdb.nrel.gov/) hosted by NREL on a Amazon Web Services (AWS) cloud through the h5py module To access large datasets, an API key can be requested from NREL. Instructions on how to set up the API key can be found here : https://github.com/NREL/hsds-examples/blob/master/notebooks/03_NSRDB_introduction.ipynb. Download data at any latitude longitude (globally) or state-county (because of repetition of county names) pairs within the US. Skim and fetch data which match different specifications, e.g. wind data for collection point at the highest elevation in the county. While HSDS allows you to splice datasets, the script allows you to also find means within ranges. Arrange data in a dataframe for multiscale analysis, with the temporal indices as tuples. [Can be saved as .csv/.txt/.json/.pkl]. Here, we import solar data as dni and wind data as wind speed for most populated data point in Harris county (TX) at an hourly resolution

In [4]:
coord_houston = (29.56999969482422, -95.05999755859375)
weather_houston = pandas.read_csv('weather_houston.csv', index_col=0)
weather_houston.index = pandas.to_datetime(weather_houston.index, utc = True)


C:\Users\cacodcar\AppData\Local\Temp\ipykernel_10256\4129034848.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_houston.index = pandas.to_datetime(weather_houston.index, utc = True)


In [5]:
solar_houston = solar_power_output(data= weather_houston, coord=coord_houston, sam = 'cecmod', module_params= 'Canadian_Solar_Inc__CS5P_220M', 
                                   inverter = 'cecinverter', inverter_params= 'ABB__MICRO_0_25_I_OUTD_US_208__208V_', 
                                   temperature_params= 'open_rack_glass_glass', aoi_model= 'no_loss', ac_model= 'sandia', spectral_model= 'no_loss')

In [6]:
wind_houston = wind_power_output(data= weather_houston, roughness_length= 0.1, turbine_type= 'V100/1800', hub_height= 92, 
                                 wind_speed_model= 'logarithmic', density_model = 'ideal_gas', temperature_model = 'linear_gradient', 
                                 power_output_model = 'power_coefficient_curve', density_correction = True, obstacle_height = 0, 
                                 observation_height = 10)

### Natural gas prices

Natural gas prices are from the Henry price index at a daily temporal resolution.

The energia.make_henry_price_df function implicitly fills in gaps in data such as weekends and public holidays by repeating the last recorded prices For e.g. if the 4th of July will use the price for the 3rd of July a typical saturday and sunday will take the values for the last friday.

The stretch functionality stretches the values over the hourly temporal scale (8760) from a daily temporal scale (365), again through repetition.

Moreover, we can remove outliers usig the remove_outliers features in data_utils

In [7]:
ng_price_df = pandas.concat(
    [make_henry_price_df(file_name='Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2016 +i, stretch= True) for i in range(5)])

### Demand data

Get [hourly power demand data](https://www.ercot.com/gridinfo/load/load_hist)  for Houston from ERCOT Coastal region

In [8]:
ercot = pandas.read_csv('ercot.csv', index_col=0)
ercot.index = pandas.to_datetime(ercot.index, utc = True)
ercot.loc['2016-11-06 23:00:00'] = ercot.loc['2016-11-06 22:00:00'] #random missing value

C:\Users\cacodcar\AppData\Local\Temp\ipykernel_10256\2618497395.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ercot.index = pandas.to_datetime(ercot.index, utc = True)


### Technology cost

NREL [Annual Technology Baseline (ATB)](https://atb.nrel.gov/) is a good source for technology cost and their expected trajectories. The annualy updated list categorizes trajectories based on the appetite for research and policy push and proposes three distinct scenarios:

- Advanced
- Moderate
- Conservative

The data for technologies not covered by ATB are drawn from literature. The trajectories for these technologies (mostly conventional) are set with only modest reduction rates as these have already stagnated in terms of cost, having witness generations of utility scale application; e.g. steam methane reforming (SMR). 

In [9]:
url = 'https://oedi-data-lake.s3.amazonaws.com/ATB/electricity/parquet/2022/ATBe.parquet'
raw_data = pandas.read_parquet(url)
raw_data = raw_data.astype(
    dtype={
        'core_metric_key': 'string',
        'core_metric_parameter': 'string',
        'core_metric_case': 'string',
        'crpyears': 'string',
        'technology': 'string',
        'technology_alias': 'string',
        'techdetail': 'string',
        'display_name': 'string',
        'scenario': 'string',
        'units': 'string'
    })


In [10]:
raw_data['technology'].unique()

<StringArray>
[                'NaturalGas_FE',    'Commercial Battery Storage',
     'Pumped Storage Hydropower',   'Residential Battery Storage',
 'Utility-Scale Battery Storage',                    'Hydropower',
                      'Biopower',                           'CSP',
                        'CommPV',               'DistributedWind',
                    'Geothermal',                 'LandbasedWind',
                  'OffShoreWind',                         'ResPV',
                     'UtilityPV', 'Utility-Scale PV-Plus-Battery',
                       'Nuclear',                       'Coal_FE',
                           'AEO']
Length: 19, dtype: string

In [11]:
def atb_gttr(core_metric_parameters, core_metric_case, crpyear, technology, techdetail, scenario):
    df_out = pandas.DataFrame()
    for i in core_metric_parameters:
        df = pandas.DataFrame(raw_data[
            (raw_data.core_metric_parameter == i) &
            (raw_data.core_metric_case == core_metric_case) &
            (raw_data.crpyears == str(crpyear)) &
            (raw_data.technology == technology) &
            (raw_data.techdetail == techdetail) &
            (raw_data.scenario == scenario)
        ][['value']])
        df = df.rename({'value': i}, axis='columns')
        df = df.reset_index()
        df_out = pandas.concat([df_out, df], axis='columns')
        df_out = df_out.loc[:, ~df_out.columns.duplicated()].copy()
        df_out = df_out.fillna(0)
        df_out = df_out.drop(columns=['index'])
    return df_out


In [12]:

hig_trl_adv = [(1 - i/(31*15)) for i in range(31)]
hig_trl_mod = [(1 - i/(31*10)) for i in range(31)]
hig_trl_con = [(1 - i/(31*5)) for i in range(31)]

med_trl_adv = [(1 - i/(31*40)) for i in range(31)]
med_trl_mod = [(1 - i/(31*30)) for i in range(31)]
med_trl_con = [(1 - i/(31*20)) for i in range(31)]

low_trl_adv = [(1 - i/(31*70)) for i in range(31)]
low_trl_mod = [(1 - i/(31*50)) for i in range(31)]
low_trl_con = [(1 - i/(31*30)) for i in range(31)]

hig_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_adv, 'Fixed O&M': hig_trl_adv, 'Variable O&M': hig_trl_adv})
hig_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_mod, 'Fixed O&M': hig_trl_mod, 'Variable O&M': hig_trl_mod})
hig_trl_con_df = pandas.DataFrame(
    data={'CAPEX': hig_trl_con, 'Fixed O&M': hig_trl_con, 'Variable O&M': hig_trl_con})
med_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': med_trl_adv, 'Fixed O&M': med_trl_adv, 'Variable O&M': med_trl_adv})
med_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': med_trl_mod, 'Fixed O&M': med_trl_mod, 'Variable O&M': med_trl_mod})
med_trl_con_df = pandas.DataFrame(
    data={'CAPEX': med_trl_con, 'Fixed O&M': med_trl_con, 'Variable O&M': med_trl_con})
low_trl_adv_df = pandas.DataFrame(
    data={'CAPEX': low_trl_adv, 'Fixed O&M': low_trl_adv, 'Variable O&M': low_trl_adv})
low_trl_mod_df = pandas.DataFrame(
    data={'CAPEX': low_trl_mod, 'Fixed O&M': low_trl_mod, 'Variable O&M': low_trl_mod})
low_trl_con_df = pandas.DataFrame(
    data={'CAPEX': low_trl_con, 'Fixed O&M': low_trl_con, 'Variable O&M': low_trl_con})
constant_df = pandas.DataFrame(
    data={'CAPEX': [1]*31, 'Fixed O&M': [1]*31, 'Variable O&M': [1]*31})


param_list = ['CAPEX', 'Fixed O&M', 'Variable O&M']
advanced_dict = {
    'RPV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='ResPV', techdetail='Class5', scenario='Advanced'),
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Advanced'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Advanced'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Advanced'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Advanced'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Advanced'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_adv_df,
    'SMRH': hig_trl_adv_df,
    'AqOff': hig_trl_adv_df,
    'DAC': low_trl_adv_df,
    'PEM': low_trl_adv_df,
    'H2_Comp': med_trl_adv_df,
}


moderate_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Moderate'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Moderate'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Moderate'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Moderate'),
    'PSH_discharge': constant_df,
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Moderate'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_mod_df,
    'SMRH': hig_trl_mod_df,
    'AqOff': hig_trl_mod_df,
    'DAC': low_trl_mod_df,
    'PEM': low_trl_mod_df,
    'H2_Comp': med_trl_mod_df,
}


conservative_dict = {
    'PV': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='UtilityPV', techdetail='Class1', scenario='Conservative'),
    'WF': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='LandbasedWind', techdetail='Class1', scenario='Conservative'),
    'LiI': atb_gttr(core_metric_parameters=param_list, core_metric_case='R&D', crpyear=30, technology='Commercial Battery Storage', techdetail='8Hr Battery Storage', scenario='Conservative'),
    'LII_discharge': constant_df,
    'PSH': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Pumped Storage Hydropower', techdetail='NatlClass10', scenario='Conservative'),
    'PSH_discharge': constant_df,
    # no cons, adv
    'ASMR': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='NuclearSMR', scenario='Moderate'),
    'NPP': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='Nuclear', techdetail='Nuclear', scenario='Conservative'),
    'NGCC': atb_gttr(core_metric_parameters=param_list, core_metric_case='Market', crpyear=30, technology='NaturalGas_FE', techdetail='CCCCSHFrame95%', scenario='Conservative'),
    'SMR': hig_trl_con_df,
    'SMRH': hig_trl_con_df,
    'AqOff': hig_trl_con_df,
    'DAC': low_trl_con_df,
    'PEM': low_trl_con_df,
    'H2_Comp': med_trl_con_df,
}


## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [4]:
scales = TemporalScale(discretization_list=[horizon, 365, 24])


# Additional Parameters

**Big and small M**

In [15]:
bigM = 10**5
smallM = 10

**Tax credits per 45Q**

In [16]:
credit = {'DAC': {0: 31.77, 1: 31.77, 2: 31.77, 3: 31.77, 4: 31.77, 5: 50.00, 6: 50.00, 7: 50.00, 8: 50.00, 9: 50.00},
          'AqOff': {0: 20.22, 1: 20.22, 2: 20.22, 3: 20.22, 4: 20.22, 5: 35.00, 6: 35.00, 7: 35.00, 8: 35.00, 9: 35.00}
}


## Define single period model

In [18]:
def model(name: str, year:int, cost_dict:dict, credit: dict = None, pathway: str = None):
    
    if pathway is None:
        pathway = 'full'
    
    #*---------------- Resources---------------------------------

    Solar = Resource(name='Solar', cons_max=100, basis='MW', label='Solar Power')

    Wind = Resource(name='Wind', cons_max=100, basis='MW', label='Wind Power')

    Power = Resource(name='Power', basis='MW', label='Power generated')#, demand = True)

    Uranium = Resource(name='Uranium', cons_max=100,
                    price=42.70/(250/2), basis='kg', label='Uranium')

    H2 = Resource(name='H2', basis='tons', label='Hydrogen')

    H2_C = Resource(name='H2(C)', basis='tons', label='Hydrogen(C)', demand=True, revenue= 0.005)

    CO2_AQoff = Resource(name='CO2_AQoff', basis='tons',
                        label='Carbon dioxide - sequestered', sell=True)

    H2O = Resource(name='H2O', cons_max=1000,
                price=3.00, basis='tons', label='Water', block='Resource')

    CH4 = Resource(name='CH4', cons_max=1000, price=max(ng_price_df['CH4'])*1000, basis='tons',
                label='Natural gas', varying=[VaryingResource.DETERMINISTIC_PRICE])

    CO2 = Resource(name='CO2', basis='tons',
                label='Carbon dioxide', block='Resource')

    CO2_Vent = Resource(name='CO2_Vent', sell=True, basis='tons',
                        label='Carbon dioxide - Vented')

    O2 = Resource(name='O2', sell=True, basis='tons', label='Oxygen')

    CO2_DAC = Resource(name='CO2_DAC', basis='tons',
                    label='Carbon dioxide - captured')

    LiI_Power = Resource(name='LiI_Power', basis='MW',
                        label='Power in LiI', store_max=10000, store_min=smallM)

    PSH_Power = Resource(name='PSH_Power', basis='MW',
                        label='Power in PSH', store_max=10000, store_min=smallM)

    #*---------------- Processes---------------------------------


    PEM = Process(name='PEM', conversion={Power: -1, H2: 0.3537, O2: 2.8302, H2O: -3.1839}, capex= cost_dict['PEM']['CAPEX'][year]*1.55*10**6, 
                 fopex=0, vopex=0, cap_max=100, cap_min = 0, basis = 'tons', label = 'PEM' )

    # LiI = Process(name='LiI', storage=Power, capex=1302182, fopex=41432, vopex=2000,
    #             cap_max=100, cap_min=0, store_max=10000, label='Lithium-ion battery', basis='MW')


    LiI = Process(name='LiI', conversion={Power: -1, LiI_Power: 1}, capex= cost_dict['LiI']['CAPEX'][year]*1000, 
                 fopex=cost_dict['LiI']['Fixed O&M'][year]*1000, vopex=cost_dict['LiI']['Variable O&M'][year]*1000,
                store_max=10000, store_min=smallM, cap_max=bigM, cap_min=smallM, label='Lithium-ion battery', basis='MW', block='energy_storage')

    LiI_discharge = Process(name='LiI', conversion={Power: 1, LiI_Power: -1}, capex=0.1,
                            fopex=0.01, vopex=0, cap_max=bigM, cap_min=smallM,  label='Lithium-ion battery (d)', basis='MW')

    WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= cost_dict['WF']['CAPEX'][year]*1000, 
                 fopex=cost_dict['WF']['Fixed O&M'][year]*1000, vopex=cost_dict['WF']['Variable O&M'][year]*1000, land = 10800/1800,
                cap_max=100, cap_min=0, label='Wind mill array', varying=[VaryingProcess.DETERMINISTIC_CAPACITY], basis='MW')

    PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex= cost_dict['PV']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PV']['Fixed O&M'][year]*1000, vopex=cost_dict['PV']['Variable O&M'][year]*1000,  land = 10000/1800,
                cap_max=100, cap_min=0,  varying= [VaryingProcess.DETERMINISTIC_CAPACITY], label='Solar PV', basis='MW')

    H2_Comp = Process(name='H2_Comp', conversion={Power: -0.417, H2_C: 1, H2: -1}, capex= cost_dict['H2_Comp']['CAPEX'][year]*1.6 *
                 10**6, fopex= 0, vopex= 0, cap_max=100, cap_min=0,  label='Hydrogen Compression')

    SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex= cost_dict['SMRH']['CAPEX'][year]*252000, 
                 fopex=cost_dict['SMRH']['Fixed O&M'][year]*945000, vopex=cost_dict['SMRH']['Variable O&M'][year]*51.5,
                cap_max=bigM, cap_min=smallM,  label='Steam methane reforming + CCUS', block='h2_prod')

    NGCC = Process(name='NGCC', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2: 0.297 *
                0.95},capex= cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex=cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex=cost_dict['NGCC']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    NG = Process(name='NG', conversion={Power: 1, CH4: -0.108, CO2_Vent: 0.297},capex= 0.3*cost_dict['NGCC']['CAPEX'][year]*1000, 
                 fopex= 0.3*cost_dict['NGCC']['Fixed O&M'][year]*1000, vopex= 0.3*cost_dict['NGCC']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM,  label='NGCC + 95% CC', block='power_gen')


    SMR = Process(name='SMR', capex= cost_dict['SMR']['CAPEX'][year]*240000, 
                 fopex=cost_dict['SMR']['Fixed O&M'][year]*800000, vopex=cost_dict['SMR']['Variable O&M'][year]*30,  conversion={
                Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, cap_max=bigM, cap_min=0.01,  label='Steam methane reforming', block='h2_prod')


    PSH = Process(name='PSH', conversion={Power: -1, PSH_Power: 1}, capex= cost_dict['PSH']['CAPEX'][year]*1000, 
                 fopex=cost_dict['PSH']['Fixed O&M'][year]*1000, vopex=cost_dict['PSH']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM, store_min=smallM, store_max=1000, label='Pumped storage hydropower', basis='MW', block='energy_storage')

    PSH_discharge = Process(name='PSH_discharge', conversion={Power: 1, PSH_Power: -1},  capex=0.1,
                            fopex=0.01, vopex=0, cap_max=bigM, cap_min=smallM,  label='Pumped storage hydropower (d)', basis='MW')

    ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex= cost_dict['ASMR']['CAPEX'][year]*1000, 
                 fopex=cost_dict['ASMR']['Fixed O&M'][year]*1000, vopex=cost_dict['ASMR']['Variable O&M'][year]*1000, cap_max=bigM, cap_min=smallM, label='Small modular reactors (SMRs)', block='power_gen')

    AqOff = Process(name='AqOff', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex= cost_dict['AqOff']['CAPEX'][year]*552000, 
                fopex=0, vopex=cost_dict['AqOff']['Variable O&M'][year]*4.140, cap_max=bigM, cap_min=smallM,
                label='Offshore aquifer CO2 sequestration (SMR)', block='ccus')
    
    DAC = Process(name='DAC', capex= cost_dict['DAC']['CAPEX'][year]*1000, 
                fopex=cost_dict['DAC']['Fixed O&M'][year]*1000, vopex=cost_dict['DAC']['Variable O&M'][year]*1000, conversion={
                Power: -0.193, H2O: -4.048, CO2_DAC: 1}, cap_max=bigM, cap_min=smallM,  label='Direct air capture', block='ccus')

    
    if pathway == 'full':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, NGCC, AqOff, PSH, SMR, SMRH, PV}
    
    if pathway == 'green':
        
        process_set = {LiI, WF, ASMR, PEM, H2_Comp, PSH, PV}
    
    if pathway == 'blue':
         
        process_set = {LiI, WF, H2_Comp, NGCC, AqOff, PSH, SMRH, PV}
    
    if pathway == 'grey':
        
        process_set = {LiI, H2_Comp, NG, PSH, SMR}
    
    #*---------------- Location---------------------------------
    
    if credit is not None:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, 
                          price_scale_level=2, land_cost = 10000, credit = {AqOff: credit['AqOff'][year], DAC: credit['DAC'][year]})
    else:
        houston= Location(name='HO', processes=process_set, capacity_factor={PV: solar_houston[:8760*horizon], WF: wind_houston[:8760*horizon]}, price_factor={
                   CH4: ng_price_df[:8760*horizon]}, scales=scales, label='Houston (conservative)', expenditure_scale_level=0, demand_scale_level=2, capacity_scale_level=2, price_scale_level=2, land_cost = 10000)

    #*---------------- Scenario---------------------------------
    
    scenario = Scenario(name=name, network=houston, scales=scales,  expenditure_scale_level=0, scheduling_scale_level=2,
                    network_scale_level=0, demand_scale_level=1, purchase_scale_level=2, capacity_scale_level = 2, label='full_case', demand={houston: {H2_C: 0.2740}}, annualization_factor = 0.1)

    scenario_reduced =  reduce_scenario(scenario=scenario, periods=40, scale_level=1, method=Clustermethod.AHC, include = [IncludeAHC.CAPACITY, IncludeAHC.PRICE])

    
    return scenario_reduced



In [36]:
span_ = 10

{'PV':           CAPEX  Fixed O&M  Variable O&M
 0   1333.262285  22.623690           0.0
 1   1165.860545  20.556800           0.0
 2   1163.564831  20.481812           0.0
 3   1161.269117  20.413394           0.0
 4   1158.973403  20.345707           0.0
 5   1156.677689  20.278731           0.0
 6   1155.268544  20.212448           0.0
 7   1153.855021  20.146841           0.0
 8   1152.437127  20.081893           0.0
 9   1151.014868  20.017586           0.0
 10  1149.588253  19.953906           0.0
 11  1129.828670  19.716852           0.0
 12  1110.069088  19.479846           0.0
 13  1090.309506  19.242887           0.0
 14  1070.549924  19.005976           0.0
 15  1050.790341  18.769111           0.0
 16  1031.030759  18.532294           0.0
 17  1011.271177  18.295522           0.0
 18   991.511595  18.058797           0.0
 19   971.752012  17.822117           0.0
 20   951.992430  17.585483           0.0
 21   932.232848  17.348894           0.0
 22   912.473265  17.112350 

# Define multiperiod case studies

In [37]:
moderate_credit = CaseStudy(name = 'moderate_credit', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, credit = credit) for i in range(span_)])
moderate_green = CaseStudy(name = 'moderate_green', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'green') for i in range(span_)])
moderate_grey = CaseStudy(name = 'moderate_grey', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'grey') for i in range(span_)])
moderate_blue = CaseStudy(name = 'moderate_blue', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict, pathway = 'blue') for i in range(span_)])

advanced = CaseStudy(name = 'advanced', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= advanced_dict) for i in range(span_)])
conservative = CaseStudy(name = 'conservative', scenarios= [model(name = f'scenario{i}', year= i, cost_dict=  conservative_dict) for i in range(span_)])
moderate = CaseStudy(name = 'moderate', scenarios= [model(name = f'scenario{i}', year= i, cost_dict= moderate_dict) for i in range(span_)])


# Formulate MIPs

In [40]:
moderate_credit.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND, Constraints.CREDIT}, objective=Objective.COST)
advanced.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
conservative.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

moderate.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
moderate_green.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)
moderate_blue.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

moderate_grey.formulate(constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                      Constraints.RESOURCE_BALANCE, Constraints.NETWORK, Constraints.DEMAND, Constraints.LAND}, objective=Objective.COST)

constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint land process
constraint land location
constraint land network
constraint land process cost
constraint land location cost
constraint land network cost
constraint credit process
constraint credit location
constraint credit network
constraint inventory balance
constraint inventory network
constraint storage facility
constraint production facility
constraint min production facility
constraint min storage facility
constraint demand
objective cost
constraint process capex
constraint process fopex
constraint process vopex
constraint process incidental
constraint storage cost
constraint storage cost location
constraint storage cost network
constraint production mode
constraint land process
constraint land location
constraint land network
constraint land process cost

{'scenario0_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25df20c7ec0>,
 'scenario1_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25dc0dfd990>,
 'scenario2_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25dd91f63e0>,
 'scenario3_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25ddc7f2f70>,
 'scenario4_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25ddfe439c0>,
 'scenario5_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25de39344f0>,
 'scenario6_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25de70bcfe0>,
 'scenario7_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25dea83db20>,
 'scenario8_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25dedf8a570>,
 'scenario9_reduced': <pyomo.core.base.PyomoModel.ConcreteModel at 0x25df1a3b060>}

# Solve the problems

In [42]:
moderate_credit.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
advanced.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
conservative.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_green.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_blue.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')
moderate_grey.solve(solver='gurobi', print_solversteps=True, saveformat = '.pkl')



Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46306 rows, 40502 columns and 132220 nonzeros
Model fingerprint: 0xcf71155f
Variable types: 40489 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 34417 rows and 33041 columns
Presolve time: 0.18s
Presolved: 11889 rows, 7461 columns, 39913 nonzeros
Variable types: 7454 continuous, 7 integer (7 binary)

Root relaxation: objective 3.795885e+05, 7269 iterations, 0.22 seconds (0.29 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46306 rows, 40502 columns and 132220 nonzeros
Model fingerprint: 0x0ebec8ab
Variable types: 40489 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 34417 rows and 33041 columns
Presolve time: 0.18s
Presolved: 11889 rows, 7461 columns, 39913 nonzeros
Variable types: 7454 continuous, 7 integer (7 binary)

Root relaxation: objective 3.605576e+05, 7208 iterations, 0.22 seconds (0.28 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46306 rows, 40502 columns and 132220 nonzeros
Model fingerprint: 0xc6512438
Variable types: 40489 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 34417 rows and 33041 columns
Presolve time: 0.17s
Presolved: 11889 rows, 7461 columns, 39913 nonzeros
Variable types: 7454 continuous, 7 integer (7 binary)

Root relaxation: objective 3.532886e+05, 7886 iterations, 0.22 seconds (0.29 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46306 rows, 40502 columns and 132220 nonzeros
Model fingerprint: 0xfa6fe57b
Variable types: 40489 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 34417 rows and 33041 columns
Presolve time: 0.16s
Presolved: 11889 rows, 7461 columns, 39913 nonzeros
Variable types: 7454 continuous, 7 integer (7 binary)

Root relaxation: objective 3.466772e+05, 7280 iterations, 0.22 seconds (0.29 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 46306 rows, 40502 columns and 132220 nonzeros
Model fingerprint: 0x2fcdad9c
Variable types: 40489 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [4e-05, 8e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 34417 rows and 33041 columns
Presolve time: 0.19s
Presolved: 11889 rows, 7461 columns, 39913 nonzeros
Variable types: 7454 continuous, 7 integer (7 binary)

Root relaxation: objective 3.399810e+05, 7901 iterations, 0.23 seconds (0.29 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0      

...

Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23171 rows, 19289 columns and 64484 nonzeros
Model fingerprint: 0x9f26125b
Variable types: 19282 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 23171 rows and 19289 columns
Presolve time: 0.07s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.08 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.24468e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.244679242958e+06, best bound 6.244679242958e+06, gap 0.0000%


Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23171 rows, 19289 columns and 64484 nonzeros
Model fingerprint: 0x5c1e3062
Variable types: 19282 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 23171 rows and 19289 columns
Presolve time: 0.07s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.08 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.1854e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.185402806565e+06, best bound 6.185402806565e+06, gap 0.0000%


Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23171 rows, 19289 columns and 64484 nonzeros
Model fingerprint: 0xcc425c9d
Variable types: 19282 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 23171 rows and 19289 columns
Presolve time: 0.07s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.08 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.13599e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.135994506065e+06, best bound 6.135994506065e+06, gap 0.0000%


Set parameter QCPDual to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 23171 rows, 19289 columns and 64484 nonzeros
Model fingerprint: 0xdcfce566
Variable types: 19282 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e-02, 4e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 1e+05]
Presolve removed 23171 rows and 19289 columns
Presolve time: 0.09s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.08 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 6.0833e+06 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.083296826934e+06, best bound 6.083296826934e+06, gap 0.0000%


Results(name='moderate_grey_results', results={'scenario0_reduced': scenario0_reduced, 'scenario1_reduced': scenario1_reduced, 'scenario2_reduced': scenario2_reduced, 'scenario3_reduced': scenario3_reduced, 'scenario4_reduced': scenario4_reduced, 'scenario5_reduced': scenario5_reduced, 'scenario6_reduced': scenario6_reduced, 'scenario7_reduced': scenario7_reduced, 'scenario8_reduced': scenario8_reduced, 'scenario9_reduced': scenario9_reduced})

The results are discussed in the publication: https://www.mdpi.com/2227-9717/12/3/469#app1-processes-12-00469

# Cite Us

@article{kakodkar2024multiperiod,
  title={Multiperiod Modeling and Optimization of Hydrogen-Based Dense Energy Carrier Supply Chains},
  author={Kakodkar, Rahul and Allen, R Cory and Demirhan, C Doga and Fu, Xiao and Pappas, Iosif and Mutlu, Mete and Pistikopoulos, Efstratios N},
  journal={Processes},
  volume={12},
  number={3},
  pages={469},
  year={2024},
  publisher={MDPI}
}